In [57]:
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def augment_board(board):
    """Generate 8 symmetric variants of an 8x8 board."""
    variants = []
    base = board.copy()
    
    # Rotations
    for k in range(4):
        rotated = np.rot90(base, k=k)
        variants.append(rotated)
        variants.append(np.fliplr(rotated))  # Add flipped version of each rotation

    return variants


In [ ]:
data = np.load('../../data/othello_train_data.npz' ,allow_pickle=True)
X, y = data['X'], data['y']
X = np.where(X == 3, 0, X)
X_aug = []
y_aug = []

for board, label in zip(X, y):
    variants = augment_board(board)
    X_aug.extend(variants)
    y_aug.extend([label] * len(variants))

X_aug = np.array(X_aug)
y_aug = np.array(y_aug)




In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train, X_val, y_train, y_val = train_test_split(X_aug, y_aug, test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train.reshape(-1, 64)).reshape(-1, 8, 8)
X_val = scaler.transform(X_val.reshape(-1, 64)).reshape(-1, 8, 8)

In [ ]:
print("Training data shape:", X_train.shape, y_train.shape)
print("-" *10)

print(X_train[50])
print(f'This game winner was {y_train[50]}')

In [ ]:
network = keras.Sequential([
    layers.Input(shape=(8,8,1)),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),

    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    layers.Dropout(0.3),


    layers.Dense(1, activation='tanh')
    ])
network.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),loss='mse',metrics=['r2_score'])

In [247]:
early_stop = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = network.fit(X_train, y_train, epochs=200, batch_size=32,validation_split=0.2,callbacks=[early_stop])

Epoch 1/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.8499 - r2_score: 0.0254 - val_loss: 0.7105 - val_r2_score: 0.1894
Epoch 2/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.6949 - r2_score: 0.2200 - val_loss: 0.6316 - val_r2_score: 0.3155
Epoch 3/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.5982 - r2_score: 0.3627 - val_loss: 0.5690 - val_r2_score: 0.4143
Epoch 4/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.5196 - r2_score: 0.4787 - val_loss: 0.5121 - val_r2_score: 0.4976
Epoch 5/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.4708 - r2_score: 0.5465 - val_loss: 0.5147 - val_r2_score: 0.5041
Epoch 6/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4480 - r2_score: 0.5864 - val_loss: 0.4734 - val_r2_score: 0.5616
Epoch 7/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.4327 - r2_score: 0.6122 - val_loss: 0.4668 - val_r2_score: 0.5730
Epoch 8/200
1178/1178 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.4201 - r2_score: 0

In [248]:
network.evaluate(X_val, y_val)

368/368 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.4196 - r2_score: 0.6444


[0.42123308777809143, 0.643641471862793]

In [230]:
#Uncomment below to save the  model
network.save("../../models/model_v1.keras")